In [165]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pymongo as mongo
from bson.objectid import ObjectId
import cPickle as pickle
import re
from time import sleep

from bs4 import BeautifulSoup
import requests

In [2]:
client = mongo.MongoClient()
client.database_names()
db = client.project4


In [ ]:
# ISSN LOOKUP:
"http://www.issn.cc/"

# DOI LOOKUP:
"http://search.crossref.org/"

# Other LOOKUP:
'http://www.worldcat.org/advancedsearch'


In [208]:
issn = re.compile(r'\d{4}-\d{3,4}[xX]?')

doi = re.compile(r'(10[.]+[0-9]{4,}.+?[0-9a-z])(?:[A-Z])')

old = re.compile(r'\b(10[.][0-9]{4,}/\S+\d+[^A-Z])')

old = re.compile(r'\b(10[.][0-9]{4,}(?:[.][0-9]+)*/(?:(?!["&\'<>])[^\t\n\r\f\vA-Z])+)')

old = re.compile(r'\b(10[.][0-9]{4,}(?:[.][0-9]+)*/(?:(?!["&\'<>])[^ ])+)\b')







In [238]:
col_id = []
dois = []
issns = []
col_id2 = []

def doi_lookup():
    rand_col = np.random.choice(db.collection_names())
    col = db[rand_col]
    article = col.find({})[0]
    col_id.append((rand_col, article['_id']))
    try:
        res = re.search(doi, article['text'])
        d = article['text'][res.span()[0]:res.span()[1]]
        return d[:-1]
    except Exception, e:
        print "DOI not found"
        return ''

        
        
def issn_lookup():
    rand_col = np.random.choice(db.collection_names())
    col = db[rand_col]
    article = col.find({})[0]
    date = article['date']
    pages = article['pages']
    col_id2.append((rand_col, article['_id']))
    try:
        res2 = re.search(issn, article['text'])
        i = article['text'][res2.start():res2.end()]
        return date, rand_col, i
    except:
        print "ISSN not found"
        return date, rand_col, ''
    
    

for _ in range(10):
    dois.append(doi_lookup())
    issns.append(issn_lookup())
    

In [203]:
links = []
base_url = "https://www.ncbi.nlm.nih.gov/pubmed/?term={}"
doi_alt = 'http://search.crossref.org/?q={}'


for d in dois:
    d.replace('/', '%2F')
    resp = requests.get(.format(d))
    soup = BeautifulSoup(resp.text,'lxml')
    try:
        lnk = soup.find('div', class_='item-links').find('a').attrs['href']
        links.append(lnk)
    except:
        print 'Article not found'
        links.append('')
    

Article not found
Article not found
Article not found


In [239]:
issns

[(u'2016_Jun', u'Transl_Neurodegener', u'2047-9158'),
 (u'NA', u'Pediatr_Neurol_Briefs', u'1043-3155'),
 (u'2010_Nov', u'Clin_Neuropsychol', u'1385-4046'),
 (u'2012_Apr', u'Neurodegener_Dis', u'1660-2854'),
 (u'2013_Nov', u'Neurosci_J', u'2314-4262'),
 (u'2011_Jun', u'Curr_Neurol_Neurosci_Rep', u'1528-4042'),
 (u'2015_Oct', u'J_Neurosurg_Anesthesiol', u'0898-4921'),
 (u'2015_Dec', u'Otol_Neurotol', u'1531-7129'),
 (u'2013_Nov', u'Neurosci_J', u'2314-4262'),
 (u'2016_Jun', u'Brain_Cogn', u'0278-2626')]

In [240]:
dois

[u'10.1037/a0018320bne_124_1_552010-01915-015',
 u'10.4081/ni.2009.e9',
 u'10.1017/S0952523813000035',
 u'10.1002/(ISSN)1531-8249',
 u'10.1155/2013/212767',
 u'10.4103/0976-3147.112767',
 u'10.1186/1471-2202-11-S1-P160',
 u'10.1002/(ISSN)1932-846',
 u'10.1007/s13311-015-0343-0',
 u'10.1097/MAO.000000000000089200008']

In [243]:
col_id2

db['Transl_Neurodegener'].find({'_id':ObjectId('581de0abb067bc05a36442ff')})[0]

{u'Alzheimers_Dementia_Parkinson': 0.29927323622523383,
 u'Animal_Experiments': 0.0,
 u'Attention': 0.01632811990910053,
 u'BrainMapping': 0.0,
 u'Brain_Mapping': 0.0,
 u'Coding_Latex': 0.0,
 u'Dependency': 0.40470867701835217,
 u'Developmental_Disorders': 0.0,
 u'Genetics': 0.0,
 u'ImmuneSystem_Cancer': 0.0,
 u'Max_Topic_Name': u'Psychiatric_Disorder',
 u'Max_Topic_Val': 1.026289655755064,
 u'Memory': 0.0,
 u'Neurons': 0.011746403825374148,
 u'Pain_MotorFunction': 0.6786453984838712,
 u'Psychiatric_Disorder': 1.026289655755064,
 u'Sleep': 0.26705804579190395,
 u'Stroke_Aneurysm_Damage': 0.0,
 u'_id': ObjectId('581de0abb067bc05a36442ff'),
 u'date': u'2016_Jun',
 u'pages': u'12',
 u'text': u"\n==== Front\nTransl NeurodegenerTransl NeurodegenerTranslational Neurodegeneration2047-9158BioMed Central London 5910.1186/s40035-016-0059-zReviewThe recommendations of Chinese Parkinson\u2019s disease and movement disorder society consensus on therapeutic management of Parkinson\u2019s disease Che

In [ ]:
# ISSUES

# doi is duplicated:
"10.1037/neu00000692014-07201-001"

# dates:
"10.1037/neu00000692014-07201-001" # 10.1037/neu0000069(2014-07)...


# weird duplication
"10.1093/neuonc/now008now008" # 10.1093/neuonc/now008(now008)